In [ ]:
import os
# TRAIN_ABLE_FALSE=True
# if TRAIN_ABLE_FALSE:
#     os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization,LeakyReLU,PReLU,ELU,ThresholdedReLU,Concatenate
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from  keras.callbacks import EarlyStopping,ModelCheckpoint
from tqdm import tqdm_notebook
from functools import partial
import datetime
import warnings
warnings.filterwarnings('ignore')

TRAIN_OFFLINE = False


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

if TRAIN_OFFLINE:
    train = pd.read_csv('../input/train.csv', dtype={'WindSpeed': 'object'})
else:
    train = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', dtype={'WindSpeed': 'object'})
    

In [ ]:
outcomes = train[['GameId','PlayId','Yards']].drop_duplicates()

## Data Splitting

In [ ]:
X = train_basetable.copy()
yards = X.Yards

y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

X.drop(['GameId','PlayId','Yards'], axis=1, inplace=True)

scaler = StandardScaler()
X = scaler.fit_transform(X)

## Define CRPS score calculator

In [ ]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
import re
from keras.losses import binary_crossentropy
from  keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import codecs
from keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False, verbose=0):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        self.verbose = verbose
        
        if self.verbose != 0:
            print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s

In [ ]:
# Calculate CRPS score
def crps_score(y_prediction, y_valid, shape=X.shape[0]):
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_prediction, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * shape)
    crps = np.round(val_s, 6)
    
    return crps

## Bayesian Optimisation Hyperparameter Tuning

### ANN

In [ ]:
def get_nn(x_tr, y_tr, x_val, y_val, shape, u1_x8=128, u2_x8=64, u3_x8=32, d1=0.5, d2=0.5, d3=0.5):
    u1 = max(int(u1_x8 * 8), 8)
    u2 = max(int(u2_x8 * 8), 8)
    u3 = max(int(u3_x8 * 8), 8)
    
    K.clear_session()
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(u1, input_dim=X.shape[1], activation='relu')(inp)
    x = Dropout(d1)(x)
    x = BatchNormalization()(x)
    x = Dense(u2, activation='relu')(x)
    x = Dropout(d2)(x)
    x = BatchNormalization()(x)
    x = Dense(u3, activation='relu')(x)
    x = Dropout(d3)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])
    
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=0, 
                       patience=10)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                                   save_best_only=True, verbose=0, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz

    model.fit(x_tr, y_tr,callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], epochs=100, batch_size=bsz,verbose=0)
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    crps = crps_score(y_pred, y_valid, shape=shape)

    return model, crps

def fit_nn(X=X, y=y, fold=5, u1_x8=128, u2_x8=64, u3_x8=32, d1=0.5, d2=0.5, d3=0.5):
    crps_csv_nn = []  
    kfold = KFold(fold, random_state = 42, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        tr_x, tr_y = X[tr_inds], y[tr_inds]
        val_x, val_y = X[val_inds], y[val_inds]
        
        # Train NN
        _, crps_nn = get_nn(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0], 
                            u1_x8=u1_x8, u2_x8=u2_x8, u3_x8=u3_x8, d1=d1, d2=d2, d3=d3)
        crps_csv_nn.append(crps_nn)
        
    return -np.mean(crps_csv_nn)

In [ ]:
from bayes_opt import BayesianOptimization

pbounds = {
           'u1_x8': (15.9, 256.1),
           'u2_x8': (15.9, 256.1),
           'u3_x8': (15.9, 256.1),
           'd1': (0, 0.9),
           'd2': (0, 0.9),
           'd3': (0, 0.9),
          }

optimizer_nn = BayesianOptimization(
    f=fit_nn,
    pbounds=pbounds,
    verbose=2,
    random_state=42,
)

optimizer_nn.maximize(init_points=10, n_iter=50)

In [ ]:
print('Best Number of Neurons 1: ', max(int(optimizer_nn.max['params']['u1_x8'] * 8), 8))
print('Best Number of Neurons 2: ', max(int(optimizer_nn.max['params']['u2_x8'] * 8), 8))
print('Best Number of Neurons 3: ', max(int(optimizer_nn.max['params']['u3_x8'] * 8), 8))
print('Best Dropout Rate 1: ', round(optimizer_nn.max['params']['d1'], 5))
print('Best Dropout Rate 2: ', round(optimizer_nn.max['params']['d2'], 5))
print('Best Dropout Rate 3: ', round(optimizer_nn.max['params']['d3'], 5))

### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

def get_rf(x_tr, y_tr, x_val, y_val, shape, mf=0.3, msl=15, mss=8, ne=50):
    msl = int(msl)
    mss = int(mss)
    ne = int(ne)
    
    model = RandomForestRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=15, 
                                  min_samples_split=8, n_estimators=50, n_jobs=-1, random_state=42)
    model.fit(x_tr, y_tr)
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    crps = crps_score(y_pred, y_valid, shape=shape)
    
    return model, crps

def fit_rf(X=X, y=y, fold=5, mf=0.3, msl=15, mss=8, ne=50):
    crps_csv_rf = []  
    kfold = KFold(fold, random_state = 42, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        tr_x, tr_y = X[tr_inds], y[tr_inds]
        val_x, val_y = X[val_inds], y[val_inds]
        
        # Train NN
        _, crps_rf = get_rf(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0], 
                            mf=mf, msl=msl, mss=mss, ne=ne)
        crps_csv_rf.append(crps_rf)
        
    return -np.mean(crps_csv_rf)